In [21]:
import re
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from splitter import *
from retriever import *

# Định nghĩa regex pattern cho các tiêu đề (sections)
patterns = [
    r'(?<=\n)Chương \d+: .+',  # Mục lớn như "Chương 1: TỔNG QUAN VỀ BAO BÌ"
    r'(?<=\n)\d+\.\d+ .+',  # Mục nhỏ hơn như "1.1 ĐẶC ĐIỂM CHUNG"
    r'(?<=\n)\d+\.\d+\.\d+ .+',  # Mục nhỏ hơn nữa như "1.1.1 Đặc điểm chung của bao bì"
    r'(?<=\n)\d+\.\d+\.\d+\.\d+ .+'  # Mục nhỏ hơn nữa như "1.1.2.1 Tính năng chứa đựng sản phẩm"
]

# Chia tài liệu thành các đoạn
loaders = TextLoader("doc/"+"spkt.txt", encoding = 'UTF-8')
docs = [loaders.load()]
splitter = PatternSplitter(patterns)
split_docs = splitter.split_documents(docs[0])
# In các đoạn
def beautiful_print(split_docs):
    for split_doc in split_docs:
        print(" ".join([split_doc.metadata['chapter'],split_doc.metadata['section'],split_doc.metadata['subsection'],split_doc.metadata['subsubsection'],]))
        if type(split_doc.metadata['summary']) is str:
            print(split_doc.metadata['summary'][:50])
        else:
            print(split_doc.metadata['summary'])
        print("----------")
        beautiful_print(split_doc.metadata["subtree"])

         
beautiful_print(split_docs)


<class 'list'>
before sum
Limit Token: Hệ thống sẽ chạy lại


In [20]:
from util import *
from retriever import *

question = "đặc điểm của bao bì từ hộp giấy bìa dạng hộp nhóm B"
en_question = vn_2_en(question)
print(en_question)
retriever = TreeRetriever(documents=split_docs)
result = retriever.invoke(en_question)
print(result.page_content)

    characteristics of packaging from cardboard boxes type group B
Bao bì hộp từ giấy bìa: Là bao bì dạng hộp được sản xuất từ giấy bìa (giấy có định lượng lớn hơn 160 g/m²). Hộp bao gồm một hay nhiều mảnh được cấn và bế. Các đường bế tạo đường viền mảnh hộp, các đường cấn tạo đường nếp để gấp hộp, các mảnh bế sau đó có thể được gấp và dán. Các hộp sau khi gấp dán được lưu trữ và vận chuyển đến khách hàng ở thể phẳng, sau đó được dựng thành hộp trong quá trình đóng gói.
Các mảnh bế là các mảnh phẳng có các đường viền được cắt theo hình dạng trải phẳng của hộp. Các đường phân cách giữa các diện của hộp là các đường cần có nhiệm vụ tạo nếp gấp, các đường này là cơ sở cho việc dựng hộp từ khổ phẳng sang không gian ba chiều.
Do tính dễ vận chuyển, lưu trữ nên các dạng hộp hình hộp chữ nhật được sử dụng rộng rãi trong công nghiệp và cuộc sống. Ngoài bao bì dạng hình hộp chữ nhật, với đặc tính dễ định hình của giấy bìa và khả năng linh hoạt của quá trình cấn bế, và yêu cầu hấp dẫn người tiêu

In [10]:
print(result)

page_content='Xác định các thông tin cần thiết: - Thông tin về sản phẩm:
+ Kích thước và thể tích của sản phẩm bên trong
+ Sản phẩm có cần không gian để dịch chuyển, hay cứng, hoặc có thể nén được
+ Khối lượng sản phẩm
+ Cấp độ của bao bì
+ Số lượng sản phẩm trong bao bì
+ Sản phẩm là chất khí, rắn, chất lỏng, bột hay kem: Nếu là chất khí phải quan tâm đến thể tích, áp suất khối lượng riêng, nhiệt độ. Nếu là chất lỏng cần quan tâm đến độ nhớt, dính, khả năng tạo hơi, khối lượng riêng.
+ Tính chất dễ vỡ của sản phẩm và sự chống va đập cần thiết khi vận chuyển
+ Bị hỏng khi mài mòn, va đập
+ Có bị hư hỏng khi ẩm, có cần chống ẩm hay không
+ Có mùi hay không
+ Khả năng tạo khí
+ Các yêu cầu đặc biệt để bảo quản
+ Có dầu, mỡ...
+ Nhạy cảm với nhiệt độ quá cao hoặc quá thấp
+ Thay đổi màu sắc hay tính chất khi bị chiếu sáng
+ Tính chất hóa học của sản phẩm: tính axít, tính kiềm, hoạt tính hóa học...
- Thông tin về khách hàng
+ Ai là người sử dụng (phụ nữ, đàn ông, độ tuổi, nhóm người nào)
+

# SQL

In [57]:
from langchain_community.utilities import SQLDatabase

username = "postgres"
password = "12345678"
host = "localhost"
port = "5432"
database = "postgres"
postgres_uri = f"postgresql://{username}:{password}@{host}:{port}/{database}"
db = SQLDatabase.from_uri(postgres_uri)

In [76]:
import re
import os
import openai
import psycopg2
from PIL import Image
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnableLambda
from sqlalchemy.exc import ProgrammingError

openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)

def sql_gen(question):
    for i in range(5):
        write_query = create_sql_query_chain(llm, db)
        sql_code = write_query.invoke(question)
        if ": " in sql_code:
            sql_code = sql_code.strip().split(": ")[1]
        if "```sql" in sql_code:
            sql_code = sql_code.strip()[7:-4]
        if 'SQLQuery' in sql_code:
            return sql_code
        try:
            db.run(sql_code)
            break
        except ProgrammingError as e:
            print("Code gen: \n")
            print(sql_code)
            print("\nCode SQL gen bị sai, đang tiến hành gen lại")
        except Exception as e:
            print(e)
            break
    return sql_code

write_query  = RunnableLambda(sql_gen)
execute_query = QuerySQLDataBaseTool(db=db)

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}

Remember to note the photoes
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
)

answer_chain = (
    answer_prompt
    | llm
    | StrOutputParser()
)

question = "chiều ngang Bao giấy 25kg TAPX-8"

code = chain.invoke({"question": question})
print("code:", code)
pattern = r"img\\\\produces_property\\\\[^\s]+\.png"
matches = re.findall(pattern, str(code['result']))
if matches:
    matches.reverse()
    for idx, match in enumerate(matches):
        img = Image.open(match)
        img.show()
        cap = "Hình "+str(idx+1)
        code['result'] = code['result'].replace(match,cap)
try:
    db.run(code['query'])
    response =  .invoke(code)
    if "không phải là một câu lệnh SQL hợp lệ" in response:
        response = "Không có dữ liệu để truy vấn"
except Exception as e:
    print(e)
    response = "Không có dữ liệu để truy vấn"

print("Answer: "+str(response))

code: {'question': 'chiều ngang Bao giấy 25kg TAPX-8', 'query': 'SELECT "size_type", "length" \nFROM packaging_sizes \nWHERE "id" = \'BT1KH00051KP0001\' \nLIMIT 5;', 'result': "[('Chiều dài bao', '950'), ('Chiều ngang bao', '500'), ('Nẹp đáy bao', 'Chiều rộng: 60mm, Chiều dài: 620mm'), ('Túi PE', 'Chiều ngang: 520mm, Chiều dài: 1100 mm, Đáy: 5cm, Khối lượng: 30gr')]"}
Answer: Chiều ngang của bao giấy 25kg TAPX-8 là 500 mm.


In [28]:
create_sql_query_chain(llm, db).get_prompts()[0].pretty_print()

You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLR

# Routing

In [65]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.utils.math import cosine_similarity

logic_query_prompt = """You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:

Question: """
semantic_query_prompt = """Bạn là AI Chatbot của Công ty CP TM & SX Bao Bì Ánh Sáng hay BBAS. 
                    Bạn ở đây để trả lời tất cả các câu hỏi về văn hóa doanh nghiệp với các tài liệu bạn được cung cấp. 
                    Tài liệu được cung cấp: 
                    Nếu tài liệu được cung cấp là "Không có tài liệu liên quan đến câu hỏi hoặc yêu cầu này." hoặc các tài liệu được cung cấp không liên quan đến câu hỏi hoặc yêu cầu thì trả lời là \"Tôi không được cung cấp thông tin để trả lời câu hỏi này\"
                    Lưu ý: CÁC CHỈ TRẢ LỜI CÁC CÂU HỎI HOẶC YÊU CẦU ĐƯỢC HỎI, KHÔNG THỪA, KHÔNG THIẾU VÀ KHÔNG TỰ Ý TÓM TẮT HAY RÚT NGẮN CÂU TRẢ LỜI
                    Câu hỏi:"""
                    
prompt_templates = [logic_query_prompt, semantic_query_prompt]
prompt_embeddings = embeddings.embed_documents(prompt_templates)
query_embedding = embeddings.embed_query("Doanh nghiệp có bao nhiêu nhân viên")
similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
similarity

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [63]:
most_similar = prompt_templates[similarity.argmax()]
most_similar

'Bạn là AI Chatbot của Công ty CP TM & SX Bao Bì Ánh Sáng hay BBAS. \n                    Bạn ở đây để trả lời tất cả các câu hỏi về văn hóa doanh nghiệp với các tài liệu bạn được cung cấp. \n                    Tài liệu được cung cấp: \n                    Nếu tài liệu được cung cấp là "Không có tài liệu liên quan đến câu hỏi hoặc yêu cầu này." hoặc các tài liệu được cung cấp không liên quan đến câu hỏi hoặc yêu cầu thì trả lời là "Tôi không được cung cấp thông tin để trả lời câu hỏi này"\n                    Lưu ý: CÁC CHỈ TRẢ LỜI CÁC CÂU HỎI HOẶC YÊU CẦU ĐƯỢC HỎI, KHÔNG THỪA, KHÔNG THIẾU VÀ KHÔNG TỰ Ý TÓM TẮT HAY RÚT NGẮN CÂU TRẢ LỜI\n                    Câu hỏi:'